# 🚀 Serialización y Deserialización con Pydantic
Este notebook muestra cómo serializar y deserializar **objetos** usando **Pydantic**.
La serialización y deserialización son operaciones fundamentales cuando trabajamos con modelos de datos. 
- Pydantic v2 ofrece herramientas potentes para convertir entre objetos Python y formatos como JSON, diccionarios, etc.
###### https://docs.pydantic.dev/latest/concepts/serialization/

##  📌 Serialización y Deserialización
- **Serialización**: Convertir un objeto Python en un formato que se pueda almacenar o transmitir.
- **Deserialización**: Convertir un formato almacenado o transmitido en un objeto Python.

Ambos (serialización y deserialización) son procesos que permiten convertir datos entre distintos formatos para utilizarlos, almacenarlos o transmitirlos.

In [ ]:
from pydantic import BaseModel, StrictBool
from datetime import datetime
from uuid import UUID, uuid4

# Definir el modelo Pydantic
class Usuario(BaseModel):
    id: UUID
    nombre: str
    registrado_en: datetime
    activo: StrictBool = True


Normalmente, se llama **serializar** cuando se convierte de un *objeto* a un *formato de texto (como JSON)* y **deserializar** cuando se convierte de un formato de texto a un objeto. Es decir, que la acción se centra en el objeto, desde y hacia el.

In [8]:
# Crear un objeto Usuario
usuario = Usuario(id=uuid4(), nombre="Juan Pérez", registrado_en=datetime.now())

## 📌 Serialización (Objeto → JSON)
Convertimos un objeto Pydantic a un **diccionario** y **JSON**.

In [9]:
# Serialización
usuario_dict = usuario.model_dump()
print("Diccionario:", usuario_dict)
print()

usuario_json = usuario.model_dump_json()
print("JSON:", usuario_json)
print()

Diccionario: {'id': UUID('03001601-a132-46f5-877e-e81709a7e97d'), 'nombre': 'Juan Pérez', 'registrado_en': datetime.datetime(2025, 3, 2, 13, 31, 31, 672717), 'activo': True}

JSON: {"id":"03001601-a132-46f5-877e-e81709a7e97d","nombre":"Juan Pérez","registrado_en":"2025-03-02T13:31:31.672717","activo":true}



Por default, python almacena los datos de forma similar a un diccionario, por lo que la serialización a JSON es muy sencilla.
- Observese la consulta al objeto `usuario` .

In [11]:
usuario

Usuario(id=UUID('03001601-a132-46f5-877e-e81709a7e97d'), nombre='Juan Pérez', registrado_en=datetime.datetime(2025, 3, 2, 13, 31, 31, 672717), activo=True)

La respuesta es un objeto usuario de clase Usuario, con los atributos definidos en la clase.

## 📌 Deserialización (JSON → Objeto)
Convertimos un diccionario o JSON en un objeto Pydantic.
- Esto es, convertir una estructura de datos diccionario o JSON en un objeto **python**
  -  con todos los atributos y métodos definidos en la clase.
- Pero ademas tendrá todas las validaciones dadas por Pydantic.

In [12]:
import json

# Datos de prueba para deserialización
usuario_data = {
    "id": "550e8400-e29b-41d4-a716-446655440000",
    "nombre": "Tulio López",
    "registrado_en": "2024-02-29T10:00:00",
    "activo": False
}

# Deserialización desde diccionario (con doble asterisco)
usuario_obj = Usuario(**usuario_data)
print("Objeto Usuario desde dict:", usuario_obj)
print()

# Deserialización desde JSON
#  primero con librería json convierto a string
usuario_json_str = json.dumps(usuario_data)

# luego utilizo pydantic para deserializar
usuario_obj2 = Usuario.model_validate_json(usuario_json_str)

print("Objeto Usuario desde JSON:", usuario_obj2)
print()

Objeto Usuario desde dict: id=UUID('550e8400-e29b-41d4-a716-446655440000') nombre='Tulio López' registrado_en=datetime.datetime(2024, 2, 29, 10, 0) activo=False

Objeto Usuario desde JSON: id=UUID('550e8400-e29b-41d4-a716-446655440000') nombre='Tulio López' registrado_en=datetime.datetime(2024, 2, 29, 10, 0) activo=False



## 🔥 Resumen
| Acción | Método | Output |
|--------|--------|--------|
| **Serializar a diccionario** | `model_dump()` | `dict` |
| **Serializar a JSON** | `model_dump_json()` | `str` (JSON) |
| **Deserializar desde `dict`** | `Usuario(**data)` | Objeto Pydantic |
| **Deserializar desde JSON** | `model_validate_json(json_str)` | Objeto Pydantic |

## Opciones Avanzadas de Serialización
Pydantic, con model_dump() y model_dump_json(), permite opciones avanzadas para serializar objetos.
- **`exclude`**: Excluir atributos en la serialización.
- **`include`**: Incluir solo los atributos especificados.
- **`by_alias`**: Serializar usando alias en lugar de los nombres de los atributos.
- **`exclude_unset`**: Excluir atributos no establecidos.
- **`exclude_defaults`**: Excluir atributos que tienen valores predeterminados.
- **`exclude_none`**: Excluir atributos con valor `None`.
- **`exclude_hidden`**: Excluir atributos que comienzan con `_`.

### Excluir atributos
Veamos la exclusión de campos en la serialización.
- En el ejemplo excluimos un par de campos: `descripción` y `fecha_creación`.

In [13]:
from pydantic import BaseModel, Field
from datetime import datetime
from typing import List, Optional

class Tarea(BaseModel):
    id: int
    titulo: str
    completada: bool = False
    fecha_creacion: datetime
    etiquetas: List[str] = []
    descripcion: Optional[str] = None

tarea = Tarea(
    id=1,
    titulo="Aprender Pydantic",
    fecha_creacion=datetime.now(),
    etiquetas=["python", "pydantic"],
    descripcion="Estudiar serialización con Pydantic"
)

# Serialización con exclusión de campos
dict_parcial = tarea.model_dump(exclude={"descripcion", "fecha_creacion"})
print(dict_parcial)
# {'id': 1, 'titulo': 'Aprender Pydantic', 'completada': False, 'etiquetas': ['python', 'pydantic']}

{'id': 1, 'titulo': 'Aprender Pydantic', 'completada': False, 'etiquetas': ['python', 'pydantic']}


### Inclusión selectiva de campos
- En el ejemplo, incluimos solo los campos `id` y `titulo`.

In [14]:
# Inclusión selectiva de campos
dict_solo_id_titulo = tarea.model_dump(include={"id", "titulo"})
print(dict_solo_id_titulo)
# {'id': 1, 'titulo': 'Aprender Pydantic'}

{'id': 1, 'titulo': 'Aprender Pydantic'}


### Exclusión de valores nulos o por defecto

In [23]:
# Excluir valores nulos o por defecto
class Tarea(BaseModel):
    id: int
    titulo: str
    completada: bool = False
    fecha_creacion: datetime
    etiquetas: List[str] = []
    descripcion: Optional[str] = None
    campos_nulos: Optional[str] = None
    campos_por_defecto: str = "campo por defecto"
    
otra_tarea = Tarea(
    id=2,
    titulo="Otra tarea",
    fecha_creacion=datetime.now(),
    campos_nulos=None,
    campos_por_defecto="otro campo por defecto"
)
print(otra_tarea.model_dump(exclude_defaults=True))


print(otra_tarea.model_dump(exclude_none=True))

{'id': 2, 'titulo': 'Otra tarea', 'fecha_creacion': datetime.datetime(2025, 3, 2, 18, 18, 13, 401185), 'campos_por_defecto': 'otro campo por defecto'}
{'id': 2, 'titulo': 'Otra tarea', 'completada': False, 'fecha_creacion': datetime.datetime(2025, 3, 2, 18, 18, 13, 401185), 'etiquetas': [], 'campos_por_defecto': 'otro campo por defecto'}


Para el primer print con exclude_defaults=True:
+ No se imprimirán los siguientes campos:
  + completada: porque tiene el valor por defecto False
  + etiquetas: porque tiene el valor por defecto [] (lista vacía)
  + descripcion: porque tiene el valor por defecto None
  + campos_nulos: porque tiene el valor por defecto None
  + campos_por_defecto: aunque su valor actual es "otro campo por defecto", no es su valor por defecto, por lo que SÍ se imprimirá.

En el segundo print con exclude_none=True:
+ No se imprimirán los siguientes campos:
  + campos_nulos: porque su valor es None
  + descripcion: porque su valor es None
+ Se imprimirán los demás campos, aunque tengan valores por defecto.

### Personalización Avanzada de Serialización y Deserialización
Pydantic facilita la personalización de la serialización y deserialización de datos mediante la definición de métodos especiales en el modelo. 
+ A continuación, se presentan algunos de los métodos especiales más comunes que se pueden definir en un modelo Pydantic:

In [ ]:
from pydantic import BaseModel, ConfigDict
from datetime import datetime

class Configuracion(BaseModel):
    model_config = ConfigDict(
        str_strip_whitespace=True,  # Eliminar espacios en strings
        validate_default=True,      # Validar los valores por defecto
        extra="ignore",             # Ignorar campos extra
        str_to_lower=True,          # Convertir strings a minúsculas
        json_encoders={             # Encoders personalizados
            datetime: lambda dt: dt.strftime("%d-%m-%Y")
        }
    )
    
    nombre: str
    fecha: datetime
    activo: bool = True

datos = {
    "nombre": "  USUARIO  ",
    "fecha": "2025-03-01T10:00:00",
    "campo_extra": "valor ignorado"
}

config = Configuracion.model_validate(datos)
print(config.nombre)  # "usuario" (minúsculas y sin espacios)
print(config.model_dump_json())
# {"nombre":"usuario","fecha":"2025-03-01","activo":true}

usuario
{"nombre":"usuario","fecha":"01-03-2025","activo":true}


###  Alias y Serializadores Personalizados

In [26]:
from pydantic import BaseModel, Field
from datetime import datetime
from typing import Any, Dict

class Producto(BaseModel):
    id: int
    nombre: str
    # El campo se llama 'precio_venta' en Python pero 'precio' en JSON
    precio_venta: float = Field(serialization_alias="precio")
    # Excluir este campo de la serialización por defecto
    stock_interno: int = Field(exclude=True)
    # Personalizar la serialización con alias e inclusión condicional
    fecha_actualizacion: datetime = Field(
        serialization_alias="ultima_actualizacion",
        exclude=lambda _: False  # Siempre incluir en la serialización
    )

producto = Producto(
    id=101,
    nombre="Laptop",
    precio_venta=999.99,
    stock_interno=50,
    fecha_actualizacion=datetime.now()
)

print(producto.model_dump(by_alias=True))
# {'id': 101, 'nombre': 'Laptop', 'precio': 999.99, 'ultima_actualizacion': datetime(...)}
print()
print(producto.model_dump())

{'id': 101, 'nombre': 'Laptop', 'precio': 999.99}

{'id': 101, 'nombre': 'Laptop', 'precio_venta': 999.99}


### Validación y Transformación de Campos durante la Deserialización
- Pydantic permite definir métodos especiales para validar y transformar los campos durante la deserialización.

In [20]:

from pydantic import BaseModel, field_validator, model_validator, BeforeValidator, AfterValidator
from typing import Annotated

def normalizar_email(email: str) -> str:
    return email.strip().lower()

class Usuario(BaseModel):
    nombre: str
    # Transformación durante la validación
    email: Annotated[str, BeforeValidator(normalizar_email)]
    rol: str

    @field_validator('rol')
    @classmethod
    def validar_rol(cls, v):
        valid_roles = ["admin", "usuario", "editor"]
        if v.lower() not in valid_roles:
            raise ValueError(f"Rol inválido. Debe ser uno de: {valid_roles}")
        return v.lower()

    @model_validator(mode='after')
    def verificar_admin(self):
        if self.rol == "admin" and not self.email.endswith("@empresa.com"):
            raise ValueError("Los administradores deben usar email corporativo")
        return self

# Deserialización con transformaciones
datos_usuario = {
    "nombre": "Juan Pérez",
    "email": "  JUAN@Empresa.com  ",
    "rol": "ADMIN"
}

usuario = Usuario.model_validate(datos_usuario)
print(usuario.model_dump())
# {'nombre': 'Juan Pérez', 'email': 'juan@empresa.com', 'rol': 'admin'}

{'nombre': 'Juan Pérez', 'email': 'juan@empresa.com', 'rol': 'admin'}


## Serialización y deserialización de objetos anidados
### Serialización de Objetos Anidados

In [28]:
from pydantic import BaseModel
from typing import List, Dict, Optional

class Direccion(BaseModel):
    calle: str
    ciudad: str
    codigo_postal: str

class Pedido(BaseModel):
    id: int
    productos: List[str]
    cantidad: int

class Cliente(BaseModel):
    id: int
    nombre: str
    direcciones: List[Direccion]
    pedidos: List[Pedido] = []
    metadata: Dict[str, str] = {}

# Crear un cliente con datos anidados
cliente = Cliente(
    id=1,
    nombre="Elena Rodríguez",
    direcciones=[
        Direccion(calle="Calle Mayor 10", ciudad="Madrid", codigo_postal="28001"),
        Direccion(calle="Avenida Principal 5", ciudad="Barcelona", codigo_postal="08001")
    ],
    pedidos=[
        Pedido(id=101, productos=["Laptop", "Mouse"], cantidad=1),
        Pedido(id=102, productos=["Monitor"], cantidad=2)
    ],
    metadata={"preferencia": "digital", "idioma": "es"}
)

# Serialización completa con modelos anidados
datos_cliente = cliente.model_dump()
print(datos_cliente)

{'id': 1, 'nombre': 'Elena Rodríguez', 'direcciones': [{'calle': 'Calle Mayor 10', 'ciudad': 'Madrid', 'codigo_postal': '28001'}, {'calle': 'Avenida Principal 5', 'ciudad': 'Barcelona', 'codigo_postal': '08001'}], 'pedidos': [{'id': 101, 'productos': ['Laptop', 'Mouse'], 'cantidad': 1}, {'id': 102, 'productos': ['Monitor'], 'cantidad': 2}], 'metadata': {'preferencia': 'digital', 'idioma': 'es'}}


### Deserialización de datos anidados

In [29]:
# Datos anidados
datos = {
    "id": 2,
    "nombre": "Carlos Gómez",
    "direcciones": [
        {"calle": "Paseo Central 20", "ciudad": "Valencia", "codigo_postal": "46001"}
    ],
    "pedidos": [
        {"id": 201, "productos": ["Teclado"], "cantidad": 1}
    ]
}

# Deserialización de datos anidados
nuevo_cliente = Cliente.model_validate(datos)
print(nuevo_cliente)

# Acceso a los modelos anidados
print(nuevo_cliente.direcciones[0].ciudad)  # "Valencia"

id=2 nombre='Carlos Gómez' direcciones=[Direccion(calle='Paseo Central 20', ciudad='Valencia', codigo_postal='46001')] pedidos=[Pedido(id=201, productos=['Teclado'], cantidad=1)] metadata={}
Valencia


## Serialización y Deserialización de Formatos Personalizados
### Serializacion a XML

In [30]:
from pydantic import BaseModel
import xml.etree.ElementTree as ET
from datetime import datetime

class Product(BaseModel):
    id: int
    name: str
    price: float
    created_at: datetime = Field(default_factory=datetime.now)
    
    def to_xml(self):
        """Convierte el modelo a XML"""
        root = ET.Element("product")
        for field_name, field_value in self.model_dump().items():
            # Convertir valores a string para XML
            if isinstance(field_value, datetime):
                field_value = field_value.isoformat()
            
            element = ET.SubElement(root, field_name)
            element.text = str(field_value)
        
        return ET.tostring(root, encoding="unicode")

# Uso
product = Product(id=1, name="Laptop", price=999.99)
xml_data = product.to_xml()
print(xml_data)
# <product><id>1</id><name>Laptop</name><price>999.99</price><created_at>2025-03-02T12:34:56</created_at></product>

<product><id>1</id><name>Laptop</name><price>999.99</price><created_at>2025-03-02T18:41:29.105414</created_at></product>


Autor: Daniel Christello
__________________________________________________________